In [3]:
import os
import pygrib
import cfgrib
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import multiprocessing as mp

from glob import glob
from datetime import datetime, timedelta

os.environ['OMP_NUM_THREADS'] = '1'

In [4]:
nbm_dir = '/scratch/general/lustre/u1070830/nbm/'
urma_dir = '/scratch/general/lustre/u1070830/urma/'

In [9]:
def unpack_fhr(nbm_file, init=None):
    
    print(nbm_file)
    
    try:
    
        with pygrib.open(nbm_file) as grb:

            lats, lons = grb.message(1).latlons()

            msgs = grb.read()
            
            valid = datetime.strptime(
                str(msgs[0].validityDate) + '%02d'%msgs[0].validityTime, 
                '%Y%m%d%H%M')

            percentile, probability, deterministic = [], [], []
            percentile_labels, probability_labels, deterministic_labels = [], [], []

            got_deterministic = {6:False, 12:False, 24:False}

            for msg in msgs:

                interval = msg['stepRange'].split('-')
                interval = int(interval[1]) - int(interval[0])

                if 'Probability of event' in str(msg):    
                    # Probability of event above upper limit (> 0.254) NOT inclusive
                    threshold = round(msg['upperLimit']/25.4, 2)
                    probability.append([msg.values])
                    probability_labels.append([interval, threshold])

                elif 'percentileValue' in msg.keys():                
                    percentile.append([msg.values])
                    percentile_labels.append([interval, msg['percentileValue']])

                else:
                    if got_deterministic[interval] == False:
                        deterministic_labels.append(interval)
                        deterministic.append(msg.values)
                        got_deterministic[interval] = True
                    else:
                        pass

        deterministic_labels = np.array(deterministic_labels)
        deterministic_labels = deterministic_labels[np.argsort(deterministic_labels)]
        deterministic = np.array(deterministic)[np.argsort(deterministic_labels)]

        probability = np.array(probability, dtype=object).reshape(-1, lats.shape[0], lats.shape[1])
        probability_labels = np.array(probability_labels)

        percentile = np.array(percentile, dtype=object).reshape(-1, 99, lats.shape[0], lats.shape[1])
        percentile_labels = np.array(percentile_labels)

        deterministic = xr.DataArray(deterministic.astype(np.float32), name='pop',
                        dims=('interval', 'y', 'x'),
                        coords={'interval':('interval', deterministic_labels),
                                'lats':(('y', 'x'), lats), 'lons':(('y', 'x'), lons)})

        probability = xr.DataArray([probability[2:].astype(np.float32)], name='probability',
                        dims=('interval', 'threshold', 'y', 'x'),
                        coords={'interval':('interval', [24]), 'threshold':('threshold', probability_labels[2:,1]),
                                'lats':(('y', 'x'), lats), 'lons':(('y', 'x'), lons)})

        percentile = xr.DataArray(percentile.astype(np.float32), name='percentile',
                        dims=('interval', 'percentile', 'y', 'x'),
                        coords={'interval':('interval', np.unique(percentile_labels[:, 0])), 'percentile':('percentile', range(1, 100)),
                                'lats':(('y', 'x'), lats), 'lons':(('y', 'x'), lons)})

        ds = xr.Dataset()

        # ds['fhr'] = fhr
        ds['time'] = valid
        ds.attrs['InitTime'] = str(init)

        ds['qpf'] = deterministic
        ds['probx'] = probability
        ds['pqpf'] = percentile
        
    except:
        raise
        return None
    
    else:
        return ds

In [10]:
init = datetime(2020, 10, 1, 0)
nbm_flist = sorted(glob(nbm_dir + init.strftime('%Y%m%d') + '/*t%02dz*'%init.hour))

unpack_fhr(nbm_flist[8])

# with mp.get_context('fork').Pool(int(len(nbm_flist)/2)) as p:
#     returns = p.map(unpack_fhr, nbm_flist, chunksize=1)
#     p.close()
#     p.join()
    
# returns = [item for item in returns if item is not None]
# returns = xr.concat(returns)

/scratch/general/lustre/u1070830/nbm/20201001/blend.t00z.qmd.f048.WR.grib2


<xarray.Dataset>
Dimensions:     (interval: 3, percentile: 99, threshold: 9, x: 1132, y: 1051)
Coordinates:
  * interval    (interval) int64 6 12 24
    lats        (y, x) float64 26.48 26.48 26.49 26.49 ... 52.77 52.77 52.77
    lons        (y, x) float64 -128.2 -128.2 -128.1 ... -100.2 -100.2 -100.2
  * threshold   (threshold) float64 0.1 0.25 0.5 1.0 2.0 3.0 4.0 5.0 6.0
  * percentile  (percentile) int64 1 2 3 4 5 6 7 8 9 ... 92 93 94 95 96 97 98 99
Dimensions without coordinates: x, y
Data variables:
    time        datetime64[ns] 2020-10-03
    qpf         (interval, y, x) float32 0.0 0.0 0.0 0.0 ... 0.179 0.217 0.252
    probx       (interval, threshold, y, x) float32 nan nan nan ... 0.0 0.0 0.0
    pqpf        (interval, percentile, y, x) float32 0.0 0.0 0.0 ... 5.335 5.317
Attributes:
    InitTime:  None